In [2]:
import pandas as pd
import numpy as np

In [5]:
# Import the IRIS dataset
from sklearn.datasets import load_iris
dataset = load_iris()
# The dataset is stored as a dictionary.
# Print the different keys present in the dictionary
print(dataset.keys())

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])


In [6]:
print(dataset['DESCR'])

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [20]:
# Load the dataset into a Pandas dataframe
df = pd.DataFrame(data=np.c_[dataset['data'], dataset['target']],
                 columns=dataset['feature_names'] + ['class'])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [25]:
# Split dataset into one with labels and one without labels for training and testing split
X = df.drop(columns=['class'], axis=1)
y = df['class']

In [27]:
# Split dataset into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [28]:
# Use sklearn in-built KNN classifier
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [29]:
classifier.predict(X_test)

array([0., 1., 1., 0., 2., 1., 2., 0., 0., 2., 1., 0., 2., 1., 1., 0., 1.,
       1., 0., 0., 1., 1., 1., 0., 2., 1., 0., 0., 1., 2., 1., 2., 1., 2.,
       2., 0., 1., 0., 1., 2., 2., 0., 1., 2., 1., 2., 0., 0., 0., 1.])

In [33]:
classifier.score(X_test, y_test)

0.98

In [ ]:
def distance(X1, X2, power):
    

In [32]:
class KNeighborsClassifier:
    
    def __init__(self, n_neighbours=2, metric='minkowski', p=2):
        self.n_neighbours = n_neighbours
        if metric.lower() in ['manhattan', 'euclidean', 'minkowski']:    
            self.metric = metric
        else:
            raise ValueError('distance_metric can have value of Manhattan, Euclidean or Minkowski only')
        if self.metric == 'manhattan':
            self.p = 1
        elif self.metric == 'euclidean':
            self.p = 2
        else:
            if p < 1:
                raise ValueError('p value cannot be less than 1 for Minkowski distance')
            self.p = p